In [ ]:
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.nn.modules.loss import L1Loss

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn import metrics


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(0)

scaler = StandardScaler()

Mounted at /content/drive


In [ ]:
# Read Dataset
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/4105a6/Housing.csv')

def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

def trinary_map(x):
    return x.map({'unfurnished': 0, 'semi-furnished': 0.5, 'furnished' : 1})

binarymap =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
trinarymap = ['furnishingstatus']

# Standardize and turn data into numbers
df[binarymap] = df[binarymap].apply(binary_map)
df[trinarymap] = df[trinarymap].apply(trinary_map)

features = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']
dfX = df[features].values
dfT = df['price'].values
dfX = scaler.fit_transform(dfX)

#split into test/train
Xtrain, Xval, Ttrain, Tval = train_test_split(dfX, dfT, train_size = 0.8, test_size=0.2, random_state=100)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

XtrainT = torch.tensor(Xtrain, dtype=torch.float32)
TtrainT = torch.tensor(Ttrain, dtype=torch.float32)
XvalT = torch.tensor(Xval, dtype=torch.float32)
TvalT = torch.tensor(Tval, dtype=torch.float32)

num_epochs = 5000

model = SimpleNN(Xtrain.shape[1], 32, 1)
optimizer = optim.Adam(model.parameters(), lr=0.01)
lossFn = nn.MSELoss()

for epoch in range(num_epochs):
    vpredictions = model(XvalT)
    vloss = lossFn(vpredictions, TvalT.view(-1, 1))

    predictions = model(XtrainT)
    loss = lossFn(predictions, TtrainT.view(-1, 1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
      print('Epoch %d, Loss %f, Validation Loss %f' % (epoch, float(loss), float(vloss)))

nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')

Epoch 499, Loss 26418771656704.000000, Validation Loss 25097639297024.000000
Epoch 999, Loss 26240620691456.000000, Validation Loss 24911319924736.000000
Epoch 1499, Loss 25965212205056.000000, Validation Loss 24622537900032.000000
Epoch 1999, Loss 25609094823936.000000, Validation Loss 24249249038336.000000
Epoch 2499, Loss 25185377845248.000000, Validation Loss 23805707681792.000000
Epoch 2999, Loss 24704005963776.000000, Validation Loss 23302533808128.000000
Epoch 3499, Loss 24172826722304.000000, Validation Loss 22748193619968.000000
Epoch 3999, Loss 23598240628736.000000, Validation Loss 22149731450880.000000
Epoch 4499, Loss 22985721249792.000000, Validation Loss 21513124184064.000000
Epoch 4999, Loss 22340230447104.000000, Validation Loss 20843866357760.000000
Neural Network Parameter Count: 417


In [ ]:
class MultiLayerNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MultiLayerNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        input_dim = input_size

        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(input_dim, hidden_size))
            self.hidden_layers.append(nn.ReLU())
            input_dim = hidden_size

        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

XtrainT = torch.tensor(Xtrain, dtype=torch.float32)
TtrainT = torch.tensor(Ttrain, dtype=torch.float32)
XvalT = torch.tensor(Xval, dtype=torch.float32)
TvalT = torch.tensor(Tval, dtype=torch.float32)

num_epochs = 5000

model = MultiLayerNN(Xtrain.shape[1], [32, 64, 16], 1)
optimizer = optim.Adam(model.parameters(), lr=0.01)
lossFn = nn.MSELoss()

for epoch in range(num_epochs):
    vpredictions = model(XvalT)
    vloss = lossFn(vpredictions, TvalT.view(-1, 1))

    predictions = model(XtrainT)
    loss = lossFn(predictions, TtrainT.view(-1, 1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
      print('Epoch %d, Loss %f, Validation Loss %f' % (epoch, float(loss), float(vloss)))

nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')

Epoch 499, Loss 1316456759296.000000, Validation Loss 1506650882048.000000
Epoch 999, Loss 1033169207296.000000, Validation Loss 1244243689472.000000
Epoch 1499, Loss 946720342016.000000, Validation Loss 1168031350784.000000
Epoch 1999, Loss 902056706048.000000, Validation Loss 1155677945856.000000
Epoch 2499, Loss 868249763840.000000, Validation Loss 1178763919360.000000
Epoch 2999, Loss 848209641472.000000, Validation Loss 1195175575552.000000
Epoch 3499, Loss 836454973440.000000, Validation Loss 1208801689600.000000
Epoch 3999, Loss 828575121408.000000, Validation Loss 1209600704512.000000
Epoch 4499, Loss 812133908480.000000, Validation Loss 1238263136256.000000
Epoch 4999, Loss 801882177536.000000, Validation Loss 1267082985472.000000
Neural Network Parameter Count: 3553


In [ ]:
# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#Simple NN for CIFAR10
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initial Transform
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download Datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

#Model Definitions
input_size = 32 * 32 * 3  # CIFAR-10 image size
hidden_size = 512
output_size = 10  # Number of classes in CIFAR-10
model = SimpleNN(input_size, hidden_size, output_size).to(device)
lossfn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Start Time, Define Epochs
num_epochs = 300
start_time = time.time()

#Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossfn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Finish times and print
training_time = time.time() - start_time
print(f"Training Time: {training_time:.2f} seconds")

# Accuracy Stuff
model.eval()
all_predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)

        all_predictions.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
accuracy = accuracy_score(true_labels, all_predictions)
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")


#Complexity
nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')


cuda


100%|██████████| 170498071/170498071 [00:12<00:00, 13375729.82it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/300, Loss: 1.6605
Epoch 2/300, Loss: 1.4684
Epoch 3/300, Loss: 1.3717
Epoch 4/300, Loss: 1.2880
Epoch 5/300, Loss: 1.2141
Epoch 6/300, Loss: 1.1475
Epoch 7/300, Loss: 1.0851
Epoch 8/300, Loss: 1.0222
Epoch 9/300, Loss: 0.9669
Epoch 10/300, Loss: 0.9187
Epoch 11/300, Loss: 0.8701
Epoch 12/300, Loss: 0.8306
Epoch 13/300, Loss: 0.7962
Epoch 14/300, Loss: 0.7631
Epoch 15/300, Loss: 0.7267
Epoch 16/300, Loss: 0.6945
Epoch 17/300, Loss: 0.6770
Epoch 18/300, Loss: 0.6631
Epoch 19/300, Loss: 0.6220
Epoch 20/300, Loss: 0.6259
Epoch 21/300, Loss: 0.5948
Epoch 22/300, Loss: 0.5720
Epoch 23/300, Loss: 0.5584
Epoch 24/300, Loss: 0.5621
Epoch 25/300, Loss: 0.5532
Epoch 26/300, Loss: 0.5286
Epoch 27/300, Loss: 0.5168
Epoch 28/300, Loss: 0.5046
Epoch 29/300, Loss: 0.4907
Epoch 30/300, Loss: 0.4810
Epoch 31/300, Loss: 0.4741
Epoch 32/300, Loss: 0.4567
Epoch 33/300, Loss: 0.4433
Epoch 34/300, Loss: 0.4201
Ep

In [ ]:
# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Multilayer NN for CIFAR10
class MultiLayerNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MultiLayerNN, self).__init__()
        self.hidden_layers = nn.ModuleList()
        input_dim = input_size

        for hidden_size in hidden_sizes:
            self.hidden_layers.append(nn.Linear(input_dim, hidden_size))
            self.hidden_layers.append(nn.ReLU())
            input_dim = hidden_size

        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

# Initial Transform
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download Datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

#Model Definitions
input_size = 32 * 32 * 3  # CIFAR-10 image size
hidden_size = [512,1024,256]
output_size = 10  # Number of classes in CIFAR-10
model = MultiLayerNN(input_size, hidden_size, output_size).to(device)
lossfn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Start Time, Define Epochs
num_epochs = 300
start_time = time.time()

#Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lossfn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Finish times and print
training_time = time.time() - start_time
print(f"Training Time: {training_time:.2f} seconds")

# Accuracy Stuff
model.eval()
all_predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)

        all_predictions.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
accuracy = accuracy_score(true_labels, all_predictions)
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")

#Complexity
nn_complexity = sum(p.numel() for p in model.parameters())
print(f'Neural Network Parameter Count: {nn_complexity}')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.6640
Epoch 2/300, Loss: 1.4303
Epoch 3/300, Loss: 1.2944
Epoch 4/300, Loss: 1.1757
Epoch 5/300, Loss: 1.0632
Epoch 6/300, Loss: 0.9584
Epoch 7/300, Loss: 0.8746
Epoch 8/300, Loss: 0.7926
Epoch 9/300, Loss: 0.7219
Epoch 10/300, Loss: 0.6606
Epoch 11/300, Loss: 0.5976
Epoch 12/300, Loss: 0.5581
Epoch 13/300, Loss: 0.5126
Epoch 14/300, Loss: 0.4799
Epoch 15/300, Loss: 0.4630
Epoch 16/300, Loss: 0.4285
Epoch 17/300, Loss: 0.3998
Epoch 18/300, Loss: 0.3978
Epoch 19/300, Loss: 0.3701
Epoch 20/300, Loss: 0.3466
Epoch 21/300, Loss: 0.3441
Epoch 22/300, Loss: 0.3282
Epoch 23/300, Loss: 0.3175
Epoch 24/300, Loss: 0.3024
Epoch 25/300, Loss: 0.2955
Epoch 26/300, Loss: 0.2838
Epoch 27/300, Loss: 0.2760
Epoch 28/300, Loss: 0.2746
Epoch 29/300, Loss: 0.2597
Epoch 30/300, Loss: 0.2494
Epoch 31/300, Loss: 0.2634
Epoch 32/300, Loss: 0.2529
Epoch 33/300, Loss: 0.2325
Epoch 34/300, Loss: 0.2426
Epoch 35/300, L